In [86]:
from db import executeInput, execute,dropTable
import pandas as pd

# 175. Combine Two Tables

[LeetCode Link](https://leetcode.com/problems/combine-two-tables/) | **Easy** | **Topics:** Joins, Left Join

## Schema

**Table: `Person`**

| Column Name | Type    |
|-------------|---------|
| personId    | int     |
| lastName    | varchar |
| firstName   | varchar |

`personId` is the primary key (column with unique values) for this table.
This table contains information about the ID of some persons and their first and last names.

**Table: `Address`**

| Column Name | Type    |
|-------------|---------|
| addressId   | int     |
| personId    | int     |
| city        | varchar |
| state       | varchar |

`addressId` is the primary key (column with unique values) for this table.
Each row of this table contains information about the city and state of one person with ID = `PersonId`.

## Problem

Write a solution to report the first name, last name, city, and state of each person in the `Person` table. If the address of a `personId` is not present in the `Address` table, report `null` instead.

Return the result table in any order.


In [87]:
# SETUP
data = [[1, "Wang", "Allen"], [2, "Alice", "Bob"]]
person = pd.DataFrame(data, columns=["personId", "firstName", "lastName"]).astype(
    {"personId": "Int64", "firstName": "object", "lastName": "object"}
)
data = [[1, 2, "New York City", "New York"], [2, 3, "Leetcode", "California"]]
address = pd.DataFrame(data, columns=["addressId", "personId", "city", "state"]).astype(
    {"addressId": "Int64", "personId": "Int64", "city": "object", "state": "object"}
)

executeInput()

[]

In [88]:
#SQL Solution
query ="""
SELECT firstName, lastName, city, state
FROM Person p
LEFT JOIN Address a ON p.personId = a.personID
"""

pd.DataFrame(execute(query))

,0,1,2,3
0,Allen,Wang,None,None
1,Bob,Alice,New York City,New York


In [ ]:
#Cleanup
dropTable('person')
dropTable('address')

In [ ]:
#Pandas solution
import pandas as pd

def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(person, address, on="personId", how="left")
    return merged[["firstName", "lastName", "city", "state"]]
    


In [125]:
combine_two_tables(person, address)

,firstName,lastName,city,state
0,Wang,Allen,NaN,NaN
1,Alice,Bob,New York City,New York


# 181. Employees Earning More Than Their Managers

[LeetCode Link](https://leetcode.com/problems/employees-earning-more-than-their-managers/) | **Easy** | **Topics:** Database, Join

## Schema

**Table: `Employee`**

| Column Name | Type    |
|-------------|---------|
| id          | int     |
| name        | varchar |
| salary      | int     |
| managerId   | int     |

`id` is the primary key (column with unique values) for this table.
Each row of this table indicates the ID of an employee, their name, salary, and the ID of their manager.

## Problem

Write a solution to find the employees who earn more than their managers.

Return the result table in any order.

The result format is in the following example.

## Example 1

**Input:**

`Employee` table:

| id | name  | salary | managerId |
|----|-------|--------|-----------|
| 1  | Joe   | 70000  | 3         |
| 2  | Henry | 80000  | 4         |
| 3  | Sam   | 60000  | Null      |
| 4  | Max   | 90000  | Null      |

**Output:**

| Employee |
|----------|
| Joe      |

**Explanation:**
Joe is the only employee who earns more than his manager.


In [135]:
# SETUP
data = [
    [1, "Joe", 70000, 3],
    [2, "Henry", 80000, 4],
    [3, "Sam", 60000, None],
    [4, "Max", 90000, None],
]
employee = pd.DataFrame(data, columns=["id", "name", "salary", "managerId"]).astype(
    {"id": "Int64", "name": "object", "salary": "Int64", "managerId": "Int64"}
)

executeInput()

[]

In [136]:
#PostgreSQL Soltution
query="""
SELECT e1.name as Employee
FROM Employee e1
JOIN Employee e2 ON e1.managerid = e2.id
WHERE e1.salary > e2.salary
"""
pd.DataFrame(execute(query))

,0
0,Joe


In [137]:
#cleanup
dropTable('employee')

In [140]:
# Pandas Solution
import pandas as pd

def find_employees(employee: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(employee, employee, left_on='managerId', right_on="id", suffixes=("", "_manager"))
    res = merged[merged["salary"] > merged["salary_manager"]][["name"]]
    res.columns = ['Employee']
    return res

In [141]:
find_employees(employee)

,Employee
0,Joe


# 182. Duplicate Emails

[LeetCode Link](https://leetcode.com/problems/duplicate-emails/) | **Easy** | **Topics:** Database

## Schema

**Table: `Person`**

| Column Name | Type    |
|-------------|---------|
| id          | int     |
| email       | varchar |

`id` is the primary key (column with unique values) for this table.
Each row of this table contains an email. The emails will not contain uppercase letters.

## Problem

Write a solution to report all the duplicate emails. Note that it's guaranteed that the email field is not NULL.

Return the result table in any order.

The result format is in the following example.

## Example 1

**Input:**

`Person` table:

| id | email   |
|----|---------|
| 1  | a@b.com |
| 2  | c@d.com |
| 3  | a@b.com |

**Output:**

| Email   |
|---------|
| a@b.com |

**Explanation:**
a@b.com is repeated two times.


In [208]:
# Setup
data = [[1, "a@b.com"], [2, "c@d.com"], [3, "a@b.com"]]
person = pd.DataFrame(data, columns=["id", "email"]).astype(
    {"id": "Int64", "email": "object"}
)

executeInput()

[]

In [209]:
#PostgreSQL
query="""
select email from person group by email having count(*) > 1
"""

pd.DataFrame(execute(query))

,0
0,a@b.com


In [210]:
#Cleanup
dropTable('person')

In [211]:
import pandas as pd

def duplicate_emails(person: pd.DataFrame) -> pd.DataFrame:
    # df = person.groupby('email').id.count().reset_index()
    # df.columns = ['email', 'count']
    # return df[df['count'] > 1][['email']]
    df = person[person.duplicated("email", keep=False)][["email"]]
    
    return pd.DataFrame({
                        'email': df['email'].unique()
        })

In [212]:
duplicate_emails(person)

,email
0,a@b.com


# 183. Customers Who Never Order

[LeetCode Link](https://leetcode.com/problems/customers-who-never-order/) | **Easy** | **Topics:** Database, Join

## Schema

**Table: `Customers`**

| Column Name | Type    |
|-------------|---------|
| id          | int     |
| name        | varchar |

`id` is the primary key (column with unique values) for this table.
Each row of this table indicates the ID and name of a customer.

**Table: `Orders`**

| Column Name | Type |
|-------------|------|
| id          | int  |
| customerId  | int  |

`id` is the primary key (column with unique values) for this table.
`customerId` is a foreign key (reference column) of the ID from the `Customers` table.
Each row of this table indicates the ID of an order and the ID of the customer who ordered it.

## Problem

Write a solution to find all customers who never order anything.

Return the result table in any order.

The result format is in the following example.

## Example 1

**Input:**

`Customers` table:

| id | name  |
|----|-------|
| 1  | Joe   |
| 2  | Henry |
| 3  | Sam   |
| 4  | Max   |

`Orders` table:

| id | customerId |
|----|------------|
| 1  | 3          |
| 2  | 1          |

**Output:**

| Customers |
|-----------|
| Henry     |
| Max       |


In [213]:
# Setup
data = [[1, "Joe"], [2, "Henry"], [3, "Sam"], [4, "Max"]]
customers = pd.DataFrame(data, columns=["id", "name"]).astype(
    {"id": "Int64", "name": "object"}
)
data = [[1, 3], [2, 1]]
orders = pd.DataFrame(data, columns=["id", "customerId"]).astype(
    {"id": "Int64", "customerId": "Int64"}
)
executeInput()

[]

In [238]:
# pgSol
query = """
SELECT c.name as Customers
FROM customers c
LEFT JOIN orders o 
    on c.id = o.customerid
WHERE o.customerid is null
"""

In [239]:
pd.DataFrame(execute(query))

,0
0,Henry
1,Max


In [ ]:
# pandas sol
import pandas as pd

def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    # df = pd.merge(customers, orders, how='left', left_on='id', right_on='customerId')
    # df = df[df["customerId"].isna()][["name"]]
    # df.columns = ['Customers']
    df = customers[~customers['id'].isin(orders['customerId'])]
    return df[['name']].rename(columns={'name': 'Customers'})

In [252]:
find_customers(customers,orders)

,name
1,Henry
3,Max
